<a href="https://colab.research.google.com/github/VMvashisht/VehicleClassification_using_VGG16/blob/main/DL_Project_Mark1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting Google Drive for accessing the Dataset
___
 We mount google drive so that we are able to access the dataset which we have uploaded on Google Drive. This is done because we were unable to download the hosted dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Ignore this code. Do not run this for training and evaluation!!**
 ___
 This is a failed attempt at trying to import the dataset from from tensorflow_datasets, but unfortunately the dataset is lost. HTTP 404 not found!

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the Stanford Cars dataset with both train and test splits
dataset_name = 'cars196'
(train_dataset, test_dataset), info = tfds.load(name=dataset_name, split=['train', 'test'], with_info=True)

# Display dataset information
print(info)

# Preprocess and prepare the data (you may need to adjust this based on your specific use case)
def preprocess_data(example):
    image = tf.image.resize(example['image'], (224, 224))  # Resize images to the desired size
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values to [0, 1]
    label = example['label']
    return image, label

# Apply preprocessing to the datasets
batch_size = 32
train_dataset = train_dataset.map(preprocess_data)
test_dataset = test_dataset.map(preprocess_data)

# Create ImageDataGenerator for training set
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    'path/to/save/training_data',  # Change to the desired path
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',  # Adjust based on the problem type (binary, categorical, etc.)
    shuffle=True,
)

# Create ImageDataGenerator for testing set
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    'path/to/save/testing_data',  # Change to the desired path
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',  # Adjust based on the problem type (binary, categorical, etc.)
    shuffle=False,
)

# Example: Iterate through the dataset using the generators
for images, labels in train_generator:
    print(images.shape, labels.shape)
    break

for images, labels in test_generator:
    print(images.shape, labels.shape)
    break


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

DownloadError: ignored

# Preprocessing the locally stored, raw dataset
___
The dataaset is first loaded, and then pre-processed. At the end, the different pre-processed datasets are stored locally **[Training Set, Testing Set, Validation Set]** along with their labels.

In [ ]:
import numpy as np
import cv2
import os


DATASET = '/content/drive/MyDrive/DataSet/archive/'
TRAIN_IMGS = DATASET + 'cars_train/'
LABELS = DATASET + 'devkit/train_perfect_preds.txt'

print('Preparing the list of images...')
# Get the list of paths of images
img_paths = []
for img in os.listdir('/content/drive/MyDrive/DataSet/archive/cars_train/cars_train'):
    # print(img)
    img_path = os.path.join('/content/drive/MyDrive/DataSet/archive/cars_train/cars_train', img)
    img_paths.append(img_path)

print(len(img_paths))

img_list = []
for img in img_paths:
    img = cv2.imread(img)
    try:
      img = cv2.resize(img, (256, 256))
    except:
      break
    img_list.append(img)

imgs_array = np.array(img_list)
print('Done compiling list of images!')

print(len(imgs_array))

print('Creating dataset labels...')
# Initialize the list of labels
labels_list = []
# Read and extract data from the labels text file
f = open(LABELS, 'r')
for row in f:
    labels_list.append(int(row))
f.close()
# Labels in array format
labels = np.array(labels_list)
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print('Labels created!')

# Split the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    imgs_array, labels,
    test_size=0.2,
    stratify=labels
)

# Training set
# Prepare RGB values for mean normalization
R, G, B = 0, 0, 0
print('Applying mean normalization and resizing images...')
for img in X_train:
    b, g, r = cv2.mean(img)[:3]
    R += r
    G += g
    B += b

R = R / len(X_train)
G = G / len(X_train)
B = B / len(X_train)

X_train_new = []
# Apply mean normalization and random cropping
from sklearn.feature_extraction.image import extract_patches_2d
for img in X_train:
    img[:,:,0] - B
    img[:,:,1] - G
    img[:,:,2] - R

    img = extract_patches_2d(img, (224, 224), max_patches=1)[0]
    X_train_new.append(np.array(img))

X_train = np.array(X_train_new)

X_test_new = []
# Test set
for img in X_test:
    img[:,:,0] - B
    img[:,:,1] - G
    img[:,:,2] - R

    img = cv2.resize(img, (224, 224))
    X_test_new.append(np.array(img))

X_test = np.array(X_test_new)
print('Images normalized and resized!')

# Split validation and test sets
(X_val, X_test, y_val, y_test) = train_test_split(
    X_test, y_test,
    test_size=0.5,
    stratify=y_test
)

print('Saving datasets...')
# Save the preprocessed dataset
np.save('/content/drive/MyDrive/DataSet/archive/X_train', X_train)
np.save('/content/drive/MyDrive/DataSet/archive/y_train', y_train)
np.save('/content/drive/MyDrive/DataSet/archive/X_val', X_val)
np.save('/content/drive/MyDrive/DataSet/archive/y_val', y_val)
np.save('/content/drive/MyDrive/DataSet/archive/X_test', X_test)
np.save('/content/drive/MyDrive/DataSet/archive/y_test', y_test)
print('Datasets saved!')

# Network Definition, Instantiation and Model Training Approach 1: **Transfer Learning Approach**
___

In [ ]:
from keras.applications import VGG16
from keras import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Activation
from keras.regularizers import L2

# Load VGG16 pre-trained on ImageNet
print('Loading pre-trained VGG16 model...')
model = VGG16(
    include_top=False, weights='imagenet', input_shape=(224, 224, 3)
)

# Freeze the pre-trained weights
for layer in model.layers:
    layer.trainable = False

# Add a new trainable FC layers
flatten_layer = Flatten()(model.layers[-1].output)
dense_layer = Dense(4096, activation='relu', name='fc1')(flatten_layer)
dense_layer2 = Dense(4096, activation='relu', name='fc2')(x)
output_tensor = Dense(196, activation='softmax', name='predictions')(x)

# Creating Model
model = Model(inputs=model.input, outputs=output_tensor, name=vgg16_tl)

from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau

# Compile model
opt = SGD(lr=0.0001, momentum=0.9, nesterov=True)
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

# Construct callbacks
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3)
callbacks = [early_stopping, lr_scheduler]

# Load datasets
import numpy as np
print('Loading datasets...')
X_train = np.load('/content/drive/MyDrive/DataSet/archive/X_train.npy')
y_train = np.load('/content/drive/MyDrive/DataSet/archive/y_train.npy')
X_val = np.load('/content/drive/MyDrive/DataSet/archive/X_val.npy')
y_val = np.load('/content/drive/MyDrive/DataSet/archive/y_val.npy')
print('Datasets are loaded!')

# Data augmentation
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Train model
H = model.fit(
    aug.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    steps_per_epoch=len(X_train)//32,
    epochs=50,
    callbacks=callbacks,
    verbose=1
)
print('Training done!')

# Unfreeze the pre-trained weights
for layer in model.layers:
    layer.trainable = True

# Save model
from keras.utils import plot_model
plot_model(model, show_shapes=True, to_file='/content/drive/MyDrive/DataSet/archive/model.png')
model.save('/content/drive/MyDrive/DataSet/archive/best_model.h5')

# PLOT
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.figure()
for y in ['loss', 'val_loss', 'accuracy', 'val_accuracy']:
    plt.plot(np.arange(0, len(H.history[y])), H.history[y], label=y)
plt.title('Training Loss and Accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend()
plt.savefig('/content/drive/MyDrive/DataSet/archive/loss_acc_plot.jpg')
plt.close()
print('Model saved!')

Loading pre-trained VGG16 model...


Loading datasets...
Datasets are loaded!
Epoch 1/50
203/203 [==============================] - 118s 513ms/step - loss: 5019.7051 - accuracy: 0.0062 - val_loss: 6001.2642 - val_accuracy: 0.0025 - lr: 0.0100
Epoch 2/50
203/203 [==============================] - 101s 498ms/step - loss: 5035.7104 - accuracy: 0.0383 - val_loss: 6563.2705 - val_accuracy: 0.0086 - lr: 0.0100
Epoch 3/50
203/203 [==============================] - 99s 487ms/step - loss: 5072.4980 - accuracy: 0.0645 - val_loss: 7571.9482 - val_accuracy: 0.0086 - lr: 0.0100
Epoch 4/50
203/203 [==============================] - 96s 475ms/step - loss: 4942.7549 - accuracy: 0.0955 - val_loss: 8160.8374 - val_accuracy: 0.0049 - lr: 0.0100
Epoch 5/50
203/203 [==============================] - 97s 479ms/step - loss: 3743.6604 - accuracy: 0.1714 - val_loss: 6881.5762 - val_accuracy: 0.0074 - lr: 0.0050
Epoch 6/50
203/203 [==============================] - 96s 475ms/step - loss: 3176.4170 - accuracy: 0.2041 - val_loss: 6799.2769 - val_acc

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved!


# Network Definition, Instantiation and Model Training Approach 2: **Training a VGG-16 Neural Network from Scratch**
___

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.layers import Activation
from keras.regularizers import L2

def vgg16(input_shape=(224, 224, 3), num_classes=1000):
  input_tensor = Input(shape=input_shape)
  # Block 1
  x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_tensor)
  x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
  x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
  # Block 2
  x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
  x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
  # Block 3
  x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
  x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
  x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
  x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
  # Block 4
  x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
  x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
  x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
  x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
  # Block 5
  x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
  x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
  x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
  x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
  # Flatten and dense layers
  x = Flatten(name='flatten')(x)
  x = Dense(4096, activation='relu', name='fc1')(x)
  x = Dense(4096, activation='relu', name='fc2')(x)
  output_tensor = Dense(num_classes, activation='softmax', name='predictions')(x)
  # Create model
  model = Model(inputs=input_tensor, outputs=output_tensor, name='vgg16')
  return model



model = vgg16(num_classes = 196)


from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau

# Compile model
opt = SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

# Construct callbacks
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3)
callbacks = [early_stopping, lr_scheduler]

# Load datasets
import numpy as np
print('Loading datasets...')
X_train = np.load('/content/drive/MyDrive/DataSet/archive/X_train.npy')
y_train = np.load('/content/drive/MyDrive/DataSet/archive/y_train.npy')
X_val = np.load('/content/drive/MyDrive/DataSet/archive/X_val.npy')
y_val = np.load('/content/drive/MyDrive/DataSet/archive/y_val.npy')
print('Datasets are loaded!')

# Data augmentation
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Train model
H = model.fit(
    aug.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    steps_per_epoch=len(X_train)//32,
    epochs=50,
    callbacks=callbacks,
    verbose=1
)
print('Training done!')

# # Unfreeze the pre-trained weights
# for layer in model.layers:
#     layer.trainable = True

# Save model
from keras.utils import plot_model
plot_model(model, show_shapes=True, to_file='/content/drive/MyDrive/DataSet/archive/model_2.png')
model.save('/content/drive/MyDrive/DataSet/archive/best_model_2.h5')

# PLOT
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.figure()
for y in ['loss', 'val_loss', 'accuracy', 'val_accuracy']:
    plt.plot(np.arange(0, len(H.history[y])), H.history[y], label=y)
plt.title('Training Loss and Accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend()
plt.savefig('/content/drive/MyDrive/DataSet/archive/loss_acc_plot_2.jpg')
plt.close()
print('Model saved!')

Loading datasets...
Datasets are loaded!
Epoch 1/50
203/203 [==============================] - 149s 615ms/step - loss: 5.2847 - accuracy: 0.0068 - val_loss: 5.2772 - val_accuracy: 0.0086 - lr: 0.0010
Epoch 2/50
203/203 [==============================] - 115s 568ms/step - loss: 5.2781 - accuracy: 0.0083 - val_loss: 5.2760 - val_accuracy: 0.0086 - lr: 0.0010
Epoch 3/50
203/203 [==============================] - 122s 602ms/step - loss: 5.2780 - accuracy: 0.0083 - val_loss: 5.2760 - val_accuracy: 0.0086 - lr: 0.0010
Epoch 4/50
203/203 [==============================] - 117s 578ms/step - loss: 5.2772 - accuracy: 0.0082 - val_loss: 5.2753 - val_accuracy: 0.0086 - lr: 0.0010
Epoch 5/50
203/203 [==============================] - 119s 584ms/step - loss: 5.2765 - accuracy: 0.0083 - val_loss: 5.2757 - val_accuracy: 0.0086 - lr: 0.0010
Epoch 6/50
203/203 [==============================] - 117s 576ms/step - loss: 5.2750 - accuracy: 0.0083 - val_loss: 5.2751 - val_accuracy: 0.0086 - lr: 0.0010
Epoch

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved!


# Evaluation Of Both The Models
___

## Approach 1: **Transfer Learning Approach**

In [ ]:
import numpy as np
from tensorflow import keras

# Load dataset
X_test = np.load('/content/drive/MyDrive/DataSet/archive/X_test.npy')
y_test = np.load('/content/drive/MyDrive/DataSet/archive/y_test.npy')

# Load model
model = keras.models.load_model('/content/drive/MyDrive/DataSet/archive/best_model.h5')

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model accuracy is {np.round(accuracy * 100, 2)}%')

26/26 [==============================] - 15s 190ms/step - loss: 4394.7334 - accuracy: 0.0074
Model accuracy is 0.74%


## Approach 2: **Training a VGG-16 Neural Network from Scratch**

In [ ]:
import numpy as np
from tensorflow import keras

# Load dataset
X_test = np.load('/content/drive/MyDrive/DataSet/archive/X_test.npy')
y_test = np.load('/content/drive/MyDrive/DataSet/archive/y_test.npy')

# Load model
model = keras.models.load_model('/content/drive/MyDrive/DataSet/archive/best_model_2.h5')

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model accuracy is {np.round(accuracy * 100, 2)}%')

26/26 [==============================] - 544s 21s/step - loss: 5.2727 - accuracy: 0.0086
Model accuracy is 0.86%
